# Required Libraries

In [2]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 90.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 90.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/

In [ ]:
import os
import json
import torch
import chromadb
import requests
import numpy as np
from PIL import Image
from io import BytesIO
from chromadb.config import Settings
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
from transformers import CLIPVisionModel, RobertaModel, AutoTokenizer, CLIPFeatureExtractor

import warnings
warnings.filterwarnings('ignore')

2025-08-21 11:03:10.939527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755774191.245718      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755774191.336781      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

# Download Data

In [4]:
!git clone https://github.com/FaSha20/Natural-Language-Processing-Projects

Cloning into 'Natural-Language-Processing-Projects'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 110 (delta 39), reused 57 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (110/110), 8.23 MiB | 18.09 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [5]:
with open("/kaggle/working/Natural-Language-Processing-Projects/MultimodalRAG/scientists_data/scintists_data_with_text_chunks.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(len(data), "records loaded")

300 records loaded


# Build Embeddings and DATABASE

## Load Pre-trained Encoder Model

In [25]:
vision_encoder = CLIPVisionModel.from_pretrained('SajjadAyoubi/clip-fa-vision')
preprocessor = CLIPFeatureExtractor.from_pretrained('SajjadAyoubi/clip-fa-vision')
text_encoder = RobertaModel.from_pretrained('SajjadAyoubi/clip-fa-text')
tokenizer = AutoTokenizer.from_pretrained('SajjadAyoubi/clip-fa-text')

## Fine-Tuning

In [7]:
# ?????

## Encoding Functions

In [8]:
# ---- Text Encoder ----
def embed_text(text: str):
    text_embedding = text_encoder(**tokenizer(text, return_tensors='pt')).pooler_output
    return text_embedding[0]

# ---- Image Encoder ----
def embed_image(url: str):
    try:
        headers = {"User-Agent": "ImageDatasetScript/1.0 (email@gmail.com)"}
        response = requests.get(url, headers=headers, timeout=10)
        image = Image.open(BytesIO(response.content)).convert("RGB")
        image_embedding = vision_encoder(**preprocessor(image, return_tensors='pt')).pooler_output
        return image_embedding[0]
    except Exception as e:
        return None

## Build the DATABASE

In [9]:
PERSIST_DIR = "/content/chroma_scientists_db"
os.makedirs(PERSIST_DIR, exist_ok=True)

client = chromadb.PersistentClient(path=PERSIST_DIR, settings=Settings(allow_reset=True))

text_db  = client.get_or_create_collection(name="scientists_text",  metadata={"hnsw:space": "cosine"})
image_db = client.get_or_create_collection(name="scientists_image", metadata={"hnsw:space": "cosine"})

In [10]:
def upsert_pair(idx, name, text, image_url, t_emb, i_emb):
    meta = {
        "pair_id": idx,
        "name": name,
        "text": text,
        "image_url": image_url if image_url is not None else "",
    }

    if t_emb is not None:
        text_db.add(
            ids=[f"text-{idx}"],
            embeddings=[t_emb.tolist()],
            metadatas=[meta],
        )

    if i_emb is not None:
        image_db.add(
            ids=[f"img-{idx}"],
            embeddings=[i_emb.tolist()],
            metadatas=[meta],
        )

In [11]:
for idx, person in enumerate(data):
    name = person.get("name")

    text = person.get("text_chunk", "").replace("\u200c", "")
    t_emb = embed_text(text)

    image_urls = list(person.get("image", {}).values())
    img_url = None
    for url in image_urls:
        if url:
            i_emb = embed_image(url)
            if i_emb is not None:
              img_url = url
              break

    upsert_pair(idx, name, text, img_url, t_emb, i_emb)

## Retrieval Functions

In [12]:
def search_by_text(query: str, k=5):
    q_emb = embed_text(query)
    res = text_db.query(query_embeddings=[q_emb.tolist()], n_results=k, include=["metadatas", "distances"])
    return res

def search_by_image(image_url: str, k=5):
    q_emb = embed_image(image_url)
    if q_emb is None:
        return None
    res = image_db.query(query_embeddings=[q_emb.tolist()], n_results=k, include=["metadatas", "distances"])
    return res

## Test an example

In [13]:
res = search_by_text("مریم میرزاخانی کیست؟", k=7)
for meta, dist in zip(res["metadatas"][0], res["distances"][0]):
    print("name:", meta["name"], "score:", 1-dist)
    print("text:", meta["text"])
    print("image:", meta["image_url"], "\n")

name: توفیق موسیوند score: 0.6373019814491272
text: توفیق موسیوند، پزشک برجسته و مخترع قلب مصنوعی، در ۲۹ خرداد سال ۱۳۱۶ در شهر همدان متولد شد. او که متخصص جراحی قلب و عروق، مهندس پزشکی و استاد جراحی بود، به دلیل اختراع اولین قلب مصنوعی جهان شهرت یافت. موسیوند پس از ازدواج و فرزنددار شدن، در سن ۳۷ سالگی به تحصیل در رشته علوم پزشکی پرداخت و پس از آن، سرپرستی پروژه قلب مصنوعی در کلینیک کلیولند را بر عهده گرفت. او در نهایت موفق به ساخت نخستین قلب مصنوعی درون بدنی با قابلیت کنترل از راه دور شد. این دانشمند برجسته در ۱۷ دی ۱۴۰۲ دار فانی را وداع گفت.
image:  

name: شمس شریعت تربقان score: 0.6063724756240845
text: شمس شریعت تربقان، که به عنوان "پدر علم پاتولوژی آسیبشناسی" نیز شناخته میشود، در پانزدهم مهر ماه سال ۱۳۰۵ خورشیدی در تربقان کاشمر چشم به جهان گشود. او پس از فارغالتحصیلی از دانشکده پزشکی لیون با دریافت بورس دولتی فرانسه در سال ۱۳۳۳، در سال ۱۳۴۳ به عنوان آسیبشناس در دانشگاه تهران مشغول به کار شد و در سال ۱۳۵۰ به درجه استاد تمامی رسید. این پزشک و استاد برجسته، در سال ۱۳۸۵ مفتخر به دریا

In [18]:
query_url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ6SPM0z9TmVCzlG6DB9liue9UBW7TausjImNksOHPSmsIT88Y9YF6XXSzBjn0UezNzRsyKb_-2hGAxATtRyYjnJ0ygpkOcLfu1WEq28Q"
res = search_by_image(query_url, k=5)
for meta, dist in zip(res["metadatas"][0], res["distances"][0]):
    print("name:", meta["name"], "score:", 1-dist)
    print("text:", meta["text"])
    print("image:", meta["image_url"], "\n")

name: علیاکبر جلالی score: 0.8948030471801758
text: علیاکبر جلالی، که با لقب "پدر فناوری اطلاعات ایران" نیز شناخته میشود، در یکم آذر سال ۱۳۳۳ در وامرزان، از توابع استان ایران، به دنیا آمد. ایشان یک چهره برجسته در حوزههای مختلف از جمله استادی دانشگاه، مهندسی برق، پژوهشگری و تخصص در فناوری اطلاعات به شمار میرود. از جمله آثار ارزشمند ایشان میتوان به "نظریه موج چهارم" اشاره کرد. در ۲۰ اردیبهشت ۱۳۹۵، همایشی به منظور تجلیل از مقام علمی ایشان با حضور چهرههایی چون سید حسن حسینی در دانشگاه شاهرود برگزار گردید. همچنین، در ۱ آذر ۱۴۰۲، پیامی از ایشان در یوتیوب منتشر شد که در آن درباره آغاز عصر مجازی و ضرورت آمادگی برای ورود به این دوره سخن گفتند.
image: https://upload.wikimedia.org/wikipedia/commons/thumb/4/40/Aliakbar_Jalali_2015-09-15.jpg/250px-Aliakbar_Jalali_2015-09-15.jpg 

name: محسن مصطفوی score: 0.8538682460784912
text: محسن مصطفوی، معمار برجسته معاصر، در سال ۱۳۳۳ در شهر تهران متولد شد. وی سابقه ریاست درخشان در مدرسه عالی طراحی دانشگاه هاروارد را دارد و دوره ریاست او تا پایان سال تحصیلی ۱۳

# Generator

In [ ]:
os.environ["TORCH_COMPILE_DISABLE"] = "1"
model_id = "google/gemma-3-4b-it"

model = Gemma3ForConditionalGeneration.from_pretrained(model_id, device_map="auto").eval()
processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [23]:
def load_image_from_url(url, timeout=10):
    headers = {"User-Agent": "MM-RAG/1.0 (email@example.com)"}
    resp = requests.get(url, headers=headers, timeout=timeout)
    resp.raise_for_status()
    return Image.open(BytesIO(resp.content)).convert("RGB")

def build_context_block(text_hits, image_hits, max_items=10):
    """
    text_hits / image_hits: lists of dicts with keys: name, text, image_url, distance
    Returns a compact Persian context string.
    """
    bullets = []
    # Prefer text facts first, then image captions/metadata (if any)
    for h in text_hits[:max_items]:
        if h:
            bullets.append(f"- {h.get('name','?')}: {h.get('text','')}")
    for h in image_hits[:max_items]:
        if h:
            bullets.append(f"- {h.get('name','?')}: {h.get('text','')}")
    return "\n".join(bullets) if bullets else ""

In [22]:
def _postprocess_query_result(res):
    out = []
    if not res:
        return out
    md = res.get("metadatas", [[]])[0]
    ds = res.get("distances", [[]])[0]
    for m, d in zip(md, ds):
        m = dict(m) if m else {}
        m["distance"] = float(d)
        out.append(m)
    return out

def retrieve_for_text_query(query: str, k_text=5, k_image=5):
    q_emb = embed_text(query)
    t_res = text_db.query(query_embeddings=[q_emb.tolist()], n_results=k_text, include=["metadatas", "distances"])
    try:
        i_res = image_db.query(query_embeddings=[q_emb.tolist()], n_results=k_image, include=["metadatas", "distances"])
    except:
        i_res = {}
    text_hits  = _postprocess_query_result(t_res)
    image_hits = _postprocess_query_result(i_res)
    return text_hits, image_hits

def retrieve_for_image_query(image_url: str, k_text=5, k_image=5):
    q_emb = embed_image(image_url)
    if q_emb is None:
        return [], []
    try:
        t_res = text_db.query(query_embeddings=[q_emb.tolist()], n_results=k_text, include=["metadatas", "distances"])
    except:
        t_res = {}
    i_res = image_db.query(query_embeddings=[q_emb.tolist()], n_results=k_image, include=["metadatas", "distances"])
    text_hits  = _postprocess_query_result(t_res)
    image_hits = _postprocess_query_result(i_res)
    return text_hits, image_hits

In [21]:
def gemma_rag_generate(
    question_text: str,
    text_hits,
    image_hits,
    force_image: bool = True,
    max_new_tokens: int = 256,
    temperature: float = 0.9,
    top_p: float = 0.9,
):
    # Pick best retrieved image
    best_img = None
    best_img_url = None
    if image_hits:
        best = sorted(image_hits, key=lambda x: x["distance"])[0]
        if best.get("image_url"):
            try:
                best_img = load_image_from_url(best["image_url"])
                best_img_url = best["image_url"]
            except Exception:
                best_img = None

    # Build retrieval context
    context_block = build_context_block(text_hits, image_hits, max_items=10)

    # Persian system prompt
    system_prompt = (
        "تو یک دستیار چندوجهی هستی. با تکیه بر «دانش بازیابی‌شده» و در صورت وجود «تصویر»، "
        "به پرسش زیر پاسخ بده. اگر پاسخ در داده‌های بازیابی‌شده نبود، بگو «اطلاعات کافی ندارم»."
        " پاسخ را به فارسی، دقیق و کوتاه بده."
    )

    # Build user content
    user_content = []
    if best_img is not None and force_image:
        user_content.append({"type": "image", "image": best_img})
    if context_block:
        user_content.append({"type": "text", "text": context_block})
    user_content.append({"type": "text", "text": f"پرسش: {question_text}"})

    # Messages for Gemma
    messages = [
        {"role": "system", "content": [{"type": "text", "text": system_prompt}]},
        {"role": "user", "content": user_content},
    ]

    # Apply chat template
    inputs = processor.apply_chat_template(messages, add_generation_prompt=True,
                                           tokenize=True,return_dict=True,
                                           return_tensors="pt").to(model.device, dtype=torch.bfloat16)

    input_len = inputs["input_ids"].shape[-1]

    with torch.inference_mode():
        out = model.generate(**inputs, max_new_tokens=max_new_tokens,
                             do_sample=True, temperature=temperature, top_p=top_p)[0][input_len:]

    text = processor.decode(out, skip_special_tokens=True)
    return {
        "answer": text.strip(),
        "used_image_url": best_img_url,
        "context": context_block,
        "question": question_text
    }

In [20]:
def rag_qa_from_text(query: str, k_text=5, k_image=5, force_image=True):
    t_hits, i_hits = retrieve_for_text_query(query, k_text=k_text, k_image=k_image)
    return gemma_rag_generate(query, t_hits, i_hits, force_image=force_image)

def rag_qa_from_image(image_url: str, question_text: str, k_text=5, k_image=5, force_image=True):
    t_hits, i_hits = retrieve_for_image_query(image_url, k_text=k_text, k_image=k_image)
    return gemma_rag_generate(question_text, t_hits, i_hits, force_image=force_image)

In [28]:
question = "مریم میرزاخانی کیست؟"
result = rag_qa_from_text(question, k_text=6, k_image=0, force_image=False)

print("📌 پرسش:\n", result["question"])
print("📖 محتوای استفاده‌شده:\n", result["context"])
print("🤖 پاسخ:\n", result["answer"])

W0821 11:14:19.511000 36 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode
W0821 11:14:19.538000 36 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode
skipping cudagraphs due to skipping cudagraphs due to multiple devices: device(type='cuda', index=0), device(type='cuda', index=1)


📌 پرسش:
 مریم میرزاخانی کیست؟
📖 محتوای استفاده‌شده:
 - توفیق موسیوند: توفیق موسیوند، پزشک برجسته و مخترع قلب مصنوعی، در ۲۹ خرداد سال ۱۳۱۶ در شهر همدان متولد شد. او که متخصص جراحی قلب و عروق، مهندس پزشکی و استاد جراحی بود، به دلیل اختراع اولین قلب مصنوعی جهان شهرت یافت. موسیوند پس از ازدواج و فرزنددار شدن، در سن ۳۷ سالگی به تحصیل در رشته علوم پزشکی پرداخت و پس از آن، سرپرستی پروژه قلب مصنوعی در کلینیک کلیولند را بر عهده گرفت. او در نهایت موفق به ساخت نخستین قلب مصنوعی درون بدنی با قابلیت کنترل از راه دور شد. این دانشمند برجسته در ۱۷ دی ۱۴۰۲ دار فانی را وداع گفت.
- شمس شریعت تربقان: شمس شریعت تربقان، که به عنوان "پدر علم پاتولوژی آسیبشناسی" نیز شناخته میشود، در پانزدهم مهر ماه سال ۱۳۰۵ خورشیدی در تربقان کاشمر چشم به جهان گشود. او پس از فارغالتحصیلی از دانشکده پزشکی لیون با دریافت بورس دولتی فرانسه در سال ۱۳۳۳، در سال ۱۳۴۳ به عنوان آسیبشناس در دانشگاه تهران مشغول به کار شد و در سال ۱۳۵۰ به درجه استاد تمامی رسید. این پزشک و استاد برجسته، در سال ۱۳۸۵ مفتخر به دریافت جایزه اخلاق پزشکی گردید 

In [29]:
img_url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRMy6j4H0jI7hG2CJklq2WN2OwMSJAxOeiRFQ&s"
question = "این تصویر متعلق به کیست و چه دستاوردهایی دارد؟"
result = rag_qa_from_image(img_url, question, k_text=0, k_image=1, force_image=True)

print("📌 پرسش:\n", result["question"])
print("🖼️ تصویر استفاده‌شده:\n", result["used_image_url"])
print("📖 محتوای استفاده‌شده:\n", result["context"])
print("🤖 پاسخ:\n", result["answer"])

📌 پرسش:
 این تصویر متعلق به کیست و چه دستاوردهایی دارد؟
🖼️ تصویر استفاده‌شده:
 https://articles.unesco.org/sites/default/files/2022-12/COU_01_23_WA_BOX_MARYAM_MARZAKHANI%20%281%29.jpg
📖 محتوای استفاده‌شده:
 - مریم میرزاخانی: مریم میرزاخانی، ریاضیدان برجسته و استاد دانشگاه معاصر، در ۲۲ اردیبهشت سال ۱۳۵۶ در تهران متولد شد. او در دوران تحصیل خود، دو مدال طلای المپیاد جهانی ریاضی در سالهای ۱۳۷۳ و ۱۳۷۴، به ترتیب در هنگکنگ و کانادا، کسب کرد و در سال ۱۳۸۳ مدرک دکترای خود را از دانشگاه هاروارد زیر نظر کورتیس مکمولن دریافت نمود. میرزاخانی در سال ۱۳۸۷ به عنوان استاد دانشگاه استنفورد مشغول به کار شد و در سال ۱۳۹۳ به دلیل مشارکتهایش در دینامیک و هندسه سطوح ریمانی، مفتخر به دریافت مدال فیلدز گردید. متاسفانه ایشان در تاریخ ۲۳ تیر ۱۳۹۶ در تهران درگذشت.
🤖 پاسخ:
 این تصویر متعلق به مریم میرزاخانی، ریاضیدان برجسته است. او مدال‌های المپیاد جهانی ریاضی و مدال فیلدز را کسب کرده است.
